# Perfume Segmentation K-Means

## Import libraries

In [102]:
# Import libraries
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()

True

## Setup Database

In [103]:
# Read database configuration from environment variables
database_config = {
  'hostname': os.getenv('DB_HOSTNAME'),
  'port': os.getenv('DB_PORT'),
  'username': os.getenv('DB_USERNAME'),
  'password': os.getenv('DB_PASSWORD'),
  'database': os.getenv('DB_DATABASE')
}

In [104]:
# Create a connection to the database
connection_str = f"mysql+pymysql://{database_config['username']}:{database_config['password']}@{database_config['hostname']}:{database_config['port']}/{database_config['database']}"
print("Connection String:", connection_str)  # Debug the connection string
engine = create_engine(connection_str)

Connection String: mysql+pymysql://root:@10.2.18.133:3306/perfume_segmentation


In [105]:
# Load perfumes table
query = "SELECT * FROM perfumes;"
df = pd.read_sql(query, engine)

df.head()

,id,name
0,0,Bacarat
1,1,D&G
2,2,Paris Hilton
3,3,Avril
4,4,Makkah Madinah
